In [1]:
import re
import random
import numpy as np

# 1. Implement RandomizedMotifSearch
## It will take very long time to run

In [2]:
symbol_index = {'A': 0, 'C': 1, 'G': 2, 'T': 3}


def get_most_probable_k_mer(profile, text, k):
    most_probable_k_mer = ''
    max_probability = -1
    text_len = len(text)
    for i in range(text_len - k + 1):
        pattern = text[i:i + k]
        probability = get_probability(profile, pattern)
        if probability > max_probability:
            max_probability = probability
            most_probable_k_mer = pattern
    return most_probable_k_mer


def get_most_probable_k_mers(profile, dna, k):
    k_mers = []
    for text in dna:
        k_mers.append(get_most_probable_k_mer(profile, text, k))
    return k_mers


def get_probability(profile, pattern):
    pattern_len = len(pattern)
    probability = 1
    for i in range(pattern_len):
        probability *= profile[symbol_index[pattern[i]]][i]
    return probability


def get_profile_from_motifs(motifs, pseudo_count=False):
    k = len(motifs[0])
    t = len(motifs)
    count = 0.0
    if pseudo_count:
        count = 1.0
    profile = [[count]*k, [count]*k, [count]*k, [count]*k]
    for text in motifs:
        for i in range(k):
            index = symbol_index[text[i]]
            profile[index][i] += 1
    divisor = t
    if pseudo_count:
        divisor += 4
    np_profile = np.array(profile)/divisor
    profile = np_profile.tolist()
    return profile


def get_score_from_motifs(motifs):
    t = len(motifs)
    motif_len = len(motifs[0])
    score = 0
    for i in range(motif_len):
        count = {}
        best_count = 0
        for motif in motifs:
            symbol = motif[i]
            if symbol not in count.keys():
                count[symbol] = 1
            else:
                count[symbol] += 1
            if count[symbol] > best_count:
                best_count = count[symbol]
        score += (t-best_count)
    return score


def get_random_motifs(dna, k):
    text_len = len(dna[0])
    t = len(dna)
    random_motifs = []
    for i in range(t):
        random_index = random.randint(0, text_len-k)
        pattern = dna[i][random_index: random_index+k]
        random_motifs.append(pattern)
    return random_motifs


def randomized_motif_search(dna, k, t):
    text_len = len(dna[0])
    best_motifs = get_random_motifs(dna, k)
    best_score = get_score_from_motifs(best_motifs)
    current_motifs = best_motifs.copy()
    while True:
        profile = get_profile_from_motifs(current_motifs, True)
        current_motifs = get_most_probable_k_mers(profile, dna, k)
        current_score = get_score_from_motifs(current_motifs)
        # print(current_score)
        if current_score < best_score:
            best_score = current_score
            best_motifs = current_motifs
        else:
            return best_motifs, best_score


dna = []
k = 0
t = 0
with open('rosalind_ba2f.txt') as file:
    stuffs = file.readlines()
    digits = re.findall(r'\d+', stuffs[0])
    k = int(digits[0])
    t = int(digits[1])
    dna = stuffs[1:]
    char_regex = re.compile('[^a-zA-Z]')
    dna = [char_regex.sub('', line) for line in dna]


best_score = k*t
best_motifs = []
for i in range(1000):
    motifs, score = randomized_motif_search(dna, k, t)
    if best_score > score:
        best_motifs = motifs
        best_score = score
    print(i+1)
print('\n'.join(best_motifs), best_score)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


# 2. Implement GibbsSampler
## It will take very long time to run

In [3]:
symbol_index = {'A': 0, 'C': 1, 'G': 2, 'T': 3}


def get_random_motifs(dna, k):
    text_len = len(dna[0])
    t = len(dna)
    random_motifs = []
    for i in range(t):
        random_index = random.randint(0, text_len-k)
        pattern = dna[i][random_index: random_index+k]
        random_motifs.append(pattern)
    return random_motifs


def get_probability(profile, pattern):
    pattern_len = len(pattern)
    probability = 1
    for i in range(pattern_len):
        probability *= profile[symbol_index[pattern[i]]][i]
    return probability


def get_most_probable_k_mer(profile, text, k):
    most_probable_k_mer = ''
    max_probability = -1
    text_len = len(text)
    for i in range(text_len - k + 1):
        pattern = text[i:i + k]
        probability = get_probability(profile, pattern)
        if probability > max_probability:
            max_probability = probability
            most_probable_k_mer = pattern
    return most_probable_k_mer


def get_score_from_motifs(motifs):
    t = len(motifs)
    motif_len = len(motifs[0])
    score = 0
    for i in range(motif_len):
        count = {}
        best_count = 0
        for motif in motifs:
            symbol = motif[i]
            if symbol not in count.keys():
                count[symbol] = 1
            else:
                count[symbol] += 1
            if count[symbol] > best_count:
                best_count = count[symbol]
        score += (t-best_count)
    return score


def get_profile_from_motifs(motifs, pseudo_count=False):
    k = len(motifs[0])
    t = len(motifs)
    count = 0.0
    if pseudo_count:
        count = 1.0
    profile = [[count]*k, [count]*k, [count]*k, [count]*k]
    for text in motifs:
        for i in range(k):
            index = symbol_index[text[i]]
            profile[index][i] += 1
    divisor = t
    if pseudo_count:
        divisor += 4
    np_profile = np.array(profile)/divisor
    profile = np_profile.tolist()
    return profile


def get_random_pattern_from_text(profile, text, k):
    text_len = len(text)
    sum_score = 0
    motif_score = []
    for i in range(text_len - k + 1):
        pattern = text[i:i + k]
        score = get_probability(profile, pattern)
        sum_score += score
        motif_score.append((pattern, sum_score))

    rand_num = random.random() * sum_score
    for item in motif_score:
        motif = item[0]
        value = item[1]
        if rand_num <= value:
            return motif


def gibbs_sampler(dna, k, t, n):
    current_motifs = get_random_motifs(dna, k)
    best_motifs = current_motifs.copy()
    best_score = get_score_from_motifs(best_motifs)
    for count in range(n):
        ignore_index = random.randint(0, t - 1)
        current_motifs.remove(current_motifs[ignore_index])
        profile = get_profile_from_motifs(current_motifs, True)
        random_motif = get_random_pattern_from_text(profile, dna[ignore_index], k)
        # random_motif = get_most_probable_k_mer(profile, dna[ignore_index], k)
        current_motifs.insert(ignore_index, random_motif)
        current_score = get_score_from_motifs(current_motifs)
        if current_score < best_score:
            best_motifs = current_motifs
            best_score = current_score
        # print(count+1)
    return best_motifs, best_score


dna = []
k = 0
t = 0
n = 0
with open('rosalind_ba2g.txt') as file:
    stuffs = file.readlines()
    digits = re.findall(r'\d+', stuffs[0])
    k = int(digits[0])
    t = int(digits[1])
    n = int(digits[2])
    dna = stuffs[1:]
    char_regex = re.compile('[^a-zA-Z]')
    dna = [char_regex.sub('', line) for line in dna]

best_motifs = []
best_score = 10000000000
for i in range(50):
    motifs, score = gibbs_sampler(dna, k, t, n)
    if score < best_score:
        best_motifs = motifs
        best_score = score
    print(i+1, score)
print('\n'.join(best_motifs), best_score)

1 84
2 64
3 91
4 70
5 118
6 73
7 100
8 94
9 64
10 105
11 73
12 103
13 64
14 86
15 96
16 101
17 64
18 95
19 84
20 74
21 106
22 72
23 109
24 74
25 64
26 108
27 114
28 72
29 87
30 96
31 112
32 74
33 74
34 74
35 95
36 100
37 96
38 73
39 117
40 101
41 108
42 72
43 83
44 97
45 64
46 72
47 83
48 103
49 72
50 72
CATTGTGGAACTCCT
CATTGTGGAACTCCT
CAATCTCCCACGCGT
CATGAGCTACCGAGT
CAATTGTTACCGCGT
CTGGCTCTACCGCGT
CAGCATCTACCGCGT
CAATCAGAACCGCGT
CAATCTCTACCCTAT
CAACGCCTACCGCGT
ACTTCTCTACCGCGT
CAATCTGCCCCGCGT
CAATTCGTACCGCGT
CAATCTCTACGCAGT
CAATCTCTAGTTCGT
CAATCTACGCCGCGT
TAATCTCTACCGCAC
CAATCTCTACCGACA
CAATCTCTGGGGCGT
CAATCGTAACCGCGT 64


# 3. Generate the k-mer Composition of a String

In [4]:
def get_compositions_from_text(text, k):
    compositions = []
    for i in range(len(text) - k):
        compositions.append(text[i: i + k])
    return compositions


Text = ''
k = 0
with open('rosalind_ba3a.txt') as file:
    k = int(file.readline())
    Text = file.readline()
print('\n'.join(sorted(get_compositions_from_text(Text, k))))

...


# 4. Reconstruct a String from its Genome Path

In [5]:
import re

patterns = []
with open('rosalind_ba3b.txt') as file:
    patterns = file.readlines()
    char_regex = re.compile('[^a-zA-Z]')
    patterns = [char_regex.sub('', pattern) for pattern in patterns]

genome = patterns[0][0:-1]
for pattern in patterns:
    genome += pattern[-1]
print(genome)

CCAGGCAGACTTGTCGACGATAAGCAGACCAACTATAATAGGGACAGCTACCAGAAATGTAATAGGATAATCTTAGTACATACCTATTGGTGGGGCTGAGCTGAGTGACTAGTCCTCATCCGAACCTGCTAATCGGAGAGGACTAGATGACCCCCGGCCTACAGTTGTTATGATCCTACAGGTGTAACCGGGGACTAAACTAAGGAGGCCAACGTCCGAAAGGTAGCTTTGCTTTGATCAGGGCTAACGCCTTAGATTCGCGCGCCTTTGAAATGTCGACGAGCCGTAAGCTCCTGCAGTCAGCCTCTCTAACTGTTTGCTTGGGTACTCCCAACAGGACAGGGAGCGTCGTAAGACTATCTGATTAATGAAATCTTAGTGGATTCTTGAAGCTGCGCTGTAAGAACACGCTATGTGGCGCGAAAATACTAACGGCGGTACCTAGCCTAAAGATGAAAAAACCTTCATTGTAGACTTTGCTAATAGGCAGCATGAAAGCGCGGTGGTGGCCCACACGTCACGCAGCAGCGCGAAACTTTTTCGTATACGCTCAAGCCCCCTTTTTAGCTCATATTACCCAACAAGACTACGAAAATACGCTTACACACAATTTTCGATTCACCCGAAGTCCCGTCTGCGACGGAATTGAGTATTGCCAGTCCACGGGCCGCGGAGAAGCAGCCGTCACAACGTCTATAGGCAAATTAAGTAAGTTACTCATCGTTCATGACTCTGCGCCAAGTCCCCGAAAGTATAGGAAGGCTTGTCCGCGAGGAATGCAGGCGTGGCAGTAATTGTAGAGGGAAGGTCGATCTAAGCCGATCTATGCCCAGGTCACACGGTAGTCTATAGGGTAAAGGATCGTGTTTGGAGCGGAGTATAGCGAAACATTGAATTTGACAATTACGGCGAACTGATGACGCGCATGGATAAGCTATATTGGGGTCCGAGCCAGGAAACCGGGTCCCCAGGCGTAGTCGACAGGATGCTAGCAGTTAGC

# 5. Construct the Overlap Graph of a Collection of k-mers

In [6]:
patterns = []
with open('rosalind_ba3c.txt') as file:
    patterns = file.readlines()
    char_regex = re.compile('[^a-zA-Z]')
    patterns = sorted([char_regex.sub('', pattern) for pattern in patterns])

adjList = []
for i in range(len(patterns)):
    firstPattern = patterns[i]
    for j in range(len(patterns)):
        if i == j:
            continue
        secondPattern = patterns[j]
        if firstPattern[1:] == secondPattern[:-1]:
            adjList.append(f'{firstPattern} -> {secondPattern}')

print('\n'.join(adjList))

AAAAATCGGGTGCTAAGTTT -> AAAATCGGGTGCTAAGTTTG
AAAATCGGGTGCTAAGTTTG -> AAATCGGGTGCTAAGTTTGT
AAACCCTCAAAGAACTAGTG -> AACCCTCAAAGAACTAGTGA
AAACTGCCTGAGGATTTGTT -> AACTGCCTGAGGATTTGTTG
AAAGAACTAGTGACTTATCG -> AAGAACTAGTGACTTATCGG
AAAGACTTAGGGAGAACGTC -> AAGACTTAGGGAGAACGTCA
AAAGTTACCACGGACTACGC -> AAGTTACCACGGACTACGCC
AAATAGGAGCCCCCTCGACC -> AATAGGAGCCCCCTCGACCG
AAATCGGGTGCTAAGTTTGT -> AATCGGGTGCTAAGTTTGTT
AAATGCCTTTCACGGCTTGG -> AATGCCTTTCACGGCTTGGG
AAATTACTTCCAGCTACCCA -> AATTACTTCCAGCTACCCAG
AAATTAGTCATGCATCATCT -> AATTAGTCATGCATCATCTA
AACAAGGTTTTCTCTACCAG -> ACAAGGTTTTCTCTACCAGC
AACAGGATCCGTCTCCCTAA -> ACAGGATCCGTCTCCCTAAA
AACCCTCAAAGAACTAGTGA -> ACCCTCAAAGAACTAGTGAC
AACGAGTCAGGTAAATTACT -> ACGAGTCAGGTAAATTACTT
AACGTCAGCTCCCAGGATGC -> ACGTCAGCTCCCAGGATGCG
AACGTCGAGACGCTGCTAAT -> ACGTCGAGACGCTGCTAATG
AACGTTATGCCCCAGAGTAT -> ACGTTATGCCCCAGAGTATT
AACTAGGAAGTCATTGATTC -> ACTAGGAAGTCATTGATTCA
AACTAGTGACTTATCGGACG -> ACTAGTGACTTATCGGACGG
AACTCATATTCACATTACTA -> ACTCATATTCACATTACTAC
AACTGCCTGA

# 6. Construct the De Bruijn Graph of a String

In [8]:
Text = ''
k = 0
with open('rosalind_ba3d.txt') as file:
    k = int(file.readline().strip())
    Text = file.readline().strip()

adjList = {}
for i in range(len(Text) - k + 1):
    pattern = Text[i: i + k - 1]
    nextPattern = Text[i + 1: i + k]
    if pattern not in adjList.keys():
        adjList[pattern] = [nextPattern]
    else:
        adjList[pattern].append(nextPattern)


output = ''
sortedItemList = sorted(adjList.items())
for item in sortedItemList:
    value = ','.join(item[1])
    output += f'{item[0]} -> {value}\n'
print(output)

AAAAAATACGC -> AAAAATACGCA
AAAAACGCCTT -> AAAACGCCTTA
AAAAATACGCA -> AAAATACGCAC
AAAACGCCTTA -> AAACGCCTTAG
AAAAGAGTCCA -> AAAGAGTCCAC
AAAATAATTTA -> AAATAATTTAG
AAAATACGCAC -> AAATACGCACC
AAAATCCTCGG -> AAATCCTCGGA
AAAATTAATAG -> AAATTAATAGC
AAAATTGCACC -> AAATTGCACCC
AAACCCCCAAG -> AACCCCCAAGG
AAACGCCTTAG -> AACGCCTTAGA
AAACGCGTCCC -> AACGCGTCCCA
AAACGGTCGAT -> AACGGTCGATT
AAACGTGTTCT -> AACGTGTTCTG
AAAGACAGGTT -> AAGACAGGTTT
AAAGAGTCCAC -> AAGAGTCCACT
AAAGCCACCTA -> AAGCCACCTAC
AAAGCGGAGGT -> AAGCGGAGGTT
AAAGCGTAACC -> AAGCGTAACCA
AAAGCTAGCCA -> AAGCTAGCCAC
AAATAATTTAG -> AATAATTTAGA
AAATACGCACC -> AATACGCACCA
AAATAGCAGAC -> AATAGCAGACG
AAATCCTCGGA -> AATCCTCGGAT
AAATCGATATG -> AATCGATATGA
AAATCTTGTTC -> AATCTTGTTCA
AAATGAAAGCG -> AATGAAAGCGG
AAATGCTTACA -> AATGCTTACAA
AAATGGGCTAG -> AATGGGCTAGT
AAATTAATAGC -> AATTAATAGCG
AAATTGCACCC -> AATTGCACCCC
AAATTTCAGCG -> AATTTCAGCGG
AACAGACAAGA -> ACAGACAAGAC
AACAGCTAAAT -> ACAGCTAAATG
AACATCGTCTA -> ACATCGTCTAT
AACCATCCTAC -> ACCATCCTACG
A

# 7. Construct the De Bruijn Graph of a Collection of k mers

In [9]:
patterns = []
with open('rosalind_ba3e.txt') as file:
    patterns = file.readlines()
    patterns = [pattern.rstrip() for pattern in patterns]

adjList = {}
patternLen = len(patterns[0])
for pattern in patterns:
    currentPattern = pattern[:-1]
    nextPattern = pattern[1:]
    if currentPattern not in adjList.keys():
        adjList[currentPattern] = nextPattern
    else:
        adjList[currentPattern] += f',{nextPattern}'

output = ''
sortedItemList = sorted(adjList.items())
for item in sortedItemList:
    output += f'{item[0]} -> {item[1]}\n'
print(output)

AAAAAATTGGGAGAAAAAG -> AAAAATTGGGAGAAAAAGA
AAAAACGATGGCCTCGAAT -> AAAACGATGGCCTCGAATG
AAAAAGATAGTAGTAGTTC -> AAAAGATAGTAGTAGTTCT
AAAAAGGACAAGACTGCTT -> AAAAGGACAAGACTGCTTC
AAAAATTGGGAGAAAAAGA -> AAAATTGGGAGAAAAAGAT
AAAACACCAAGAGTATTTA -> AAACACCAAGAGTATTTAC
AAAACATTACTCGCTTTTC -> AAACATTACTCGCTTTTCG
AAAACGATGGCCTCGAATG -> AAACGATGGCCTCGAATGC
AAAAGACTGCTTCATCCTA -> AAAGACTGCTTCATCCTAT
AAAAGATAGTAGTAGTTCT -> AAAGATAGTAGTAGTTCTC
AAAAGCTCGCACTGCTCTG -> AAAGCTCGCACTGCTCTGA
AAAAGGACAAGACTGCTTC -> AAAGGACAAGACTGCTTCA
AAAATTGGGAGAAAAAGAT -> AAATTGGGAGAAAAAGATA
AAACAAAAAATTGGGAGAA -> AACAAAAAATTGGGAGAAA
AAACAATCTTGAGAGAAAG -> AACAATCTTGAGAGAAAGG
AAACACCAAGAGTATTTAC -> AACACCAAGAGTATTTACA
AAACACTTGACGGCATCTG -> AACACTTGACGGCATCTGA
AAACATTACTCGCTTTTCG -> AACATTACTCGCTTTTCGA
AAACGATGGCCTCGAATGC -> AACGATGGCCTCGAATGCT
AAACGGAGCACCCCCACTT -> AACGGAGCACCCCCACTTT
AAACTGACAAGCTCTTTTT -> AACTGACAAGCTCTTTTTA
AAACTTACGTACACGATCG -> AACTTACGTACACGATCGT
AAACTTCGGAGATGCACCG -> AACTTCGGAGATGCACCGG
AAACTTTGATC